# ASC School: "Physics meets Artificial Intelligence"

## Tutorial: Variational Monte Carlo with neural quantum states for the Dynamics of the transverse-field Ising model

Filippo Vicentini, EPFL

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PhilipVinc/Lectures/blob/main/2209_Munich/Dynamics.ipynb) 



In this Tutorial we will introduce the open-source package [NetKet](https://www.netket.org/), and show some of its functionalities. 
As this week has already been packed and you have already seen several advanced applications of Neural Networks Quantum States (With Roger Melko), we will give a brief overview of the fundamentals of NetKet, then move on how to determine the ground state of an Hamiltonian, and finally how to simulate the dynamics.

If you want to see a more 'slow-paced' and _in-depth_ tutorial only on determining the Ground State of an Hamiltonian, and about the different performance characteristics of different architectures, you can have a look at the tutorials on [NetKet's website](https://netket.readthedocs.io/en/latest/tutorials/gs-ising.html).

In This tutorial, we will study a Quench from the ground state of the 1-Dimensional Transverse-Field Ising Model for transverse field $ h=1 $ to transverse field $ h=2 $. 

$$ 
\mathcal{H}=\Gamma\sum_{i}\sigma_{i}^{(x)}+V\sum_{i}\sigma_{i}^{(z)}\sigma_{i+1}^{(z)}. 
$$

In the following we assume periodic boundary conditions and we will count lattice sites starting from $ 0 $, such that $ i=0,1\dots L-1 $ and $i=L=0$. 

<div class="alert alert-info">
In this tutorial we will consider only $L=10$ sites so that simulations run fast. However, you can easily scale those simulations up if you use a GPU or if you have some extra time. 
</div>

## 0. NetKet

Go to [http://netket.org](https://netket.org)

<img src="netket_web.png" style="margin-left:auto; margin-right:auto" width="700" >


### 0a. Installing Netket 

If you are executing this notebook on Colab, you will need to install NetKet (and some other dependencies). You can do so by running the following cell:

In [ ]:
#%pip install --quiet --upgrade netket==3.5 matplotlib qutip

<div class="alert alert-info">
<b> Running on GPUs: </b>

You need a Linux computer and a Nvidia/AMD Gpu, and install the correct version of Jax.
On Colab, this is already installed if you are running in a GPU instance.
    
```
pip install --upgrade -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html 'jax[cuda]'
```

### 0b. Running NetKet

<div class="alert alert-info">
We also want make to sure that this notebook is running on the cpu. 
You can edit the field by changing "cpu" to "gpu" to make it run on the GPU if you want. 
    
But you'll need to use much larger systems to see a benefit in the runtime.
For systems with less than 40 spins GPUs slow you down remarkably.
</div>

In [ ]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"

You can check that the installation was succesfull doing:

In [ ]:
import netket as nk
print(nk.__version__)

### 0c. Import some useful packages

In [ ]:
# numerical operations in the model should always use jax.numpy 
# instead of numpy because jax supports computing derivatives. 
# If you want to better understand the difference between the two, check
# https://flax.readthedocs.io/en/latest/notebooks/jax_for_the_impatient.html
import jax.numpy as jnp

# Flax is a framework to define models using jax
import flax
# we refer to `flax.linen` as `nn`. It's a repository of 
# layers, initializers and nonlinear functions.
import flax.linen as nn

import numpy as np

import matplotlib.pyplot as plt

## 1. Defining the system and  Hamiltonian

The first step in our journey consists in defining the Hamiltonian we are interested in. 
For this purpose, we first need to define the kind of degrees of freedom we are dealing with (i.e. if we have spins, bosons, fermions etc). 

This is done specifying the Hilbert space of the problem. For example, let us concentrate on a problem with 10 spins-1/2.

### 1b. Define The computational basis (Hilbert space)

When building hilbert spaces, in general, the first argument determines the size of the local basis and the latter defines how many modes you have.

In [ ]:
import netket as nk

N = 10
hi = nk.hilbert.Spin(s=1 / 2, N=N)

NetKet's Hilbert spaces define the computational basis of the calculation, and are used to label and generate elements from it. 
The standard Spin-basis implicitly selects the `z` basis and elements of that basis will be elements $ v\in\{\pm 1\}^N $.

It is possible to generate random basis elements through the function `random_state(rng, shape, dtype)`, where the first argument must be a jax RNG state (usually built with `jax.random.PRNGKey(seed)`, second is an integer or a tuple giving the shape of the samples and the last is the dtype of the generated states.

In [ ]:
import jax
hi.random_state(jax.random.PRNGKey(0), 3)

### 1c. Define the Hamiltonian

We now need to specify the Hamiltonian. For this purpose, we will use NetKet's ```LocalOperator``` (see details [here](https://www.netket.org/docs/_generated/operator/netket.operator.LocalOperator.html#netket.operator.LocalOperator)) which is the sum of arbitrary k-local operators. 

In this specifc case, we have a 1-local operator, $ \sigma^{(x)}_i $ and a 2-local operator, $ \sigma^{(z)}_i \sigma^{(z)}_j $. We then start importing the pauli operators. 

In [ ]:
from netket.operator.spin import sigmax, sigmaz 

We now take $ \Gamma=-1 $ and start defining the 1-local parts of the Hamiltonian 

In [ ]:
Gamma = -1
H = sum([Gamma*sigmax(hi,i) for i in range(N)])

Here we have used a list comprehension to (mildly) show off our ability to write one-liners, however you could have just added the terms one by one in an explicit loop instead (though you'd end up with a whopping 3 lines of code). 

We now also add the interaction terms, using the fact that NetKet automatically recognizes products of local operators as tensor products. 

In [ ]:
Gamma = -1
H = sum([Gamma*sigmax(hi,i) for i in range(N)])
print("Local terms: \n", H)

V=-1
H += sum([V*sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])
print("Local and Interaction terms: \n", H)

In general, when manipulating NetKet objects, you should always assume that you can safely operate on them like 
you would in mathematical equations, therefore you can sum and multiply them with ease.

### 1d. Special Hamiltonians

For some specific Hamiltonians with a very specific structure, such as the Ising Hamiltonian, we can use a 'custom-made' implementation that is more performant. This implementation can only represent the Ising hamiltonian, and is of course less flexible than the mechanism above which allows us to write any (Local) hamiltonian.

To use this specific implementation, you need to define the graph over which we defined the model. In our case, that is the 1D periodic chain:

In [ ]:
lattice = nk.graph.Chain(N)

In [ ]:
print(lattice)

We can also print it, or plot it in order to inspect it.

In [ ]:
lattice.draw();

And we can now build the custom Hamiltonian

In [ ]:
?nk.operator.Ising

In [ ]:
H_special = nk.operator.Ising(hi, lattice, h=1.0, J=-1.0)

In [ ]:
# And we can compare the two: if we check if they are equal, they are not, 
# because they are different implementations
H_special == H

In [ ]:
# But if we compare their matrix versions, they do match:
import numpy as np
np.allclose(H_special.to_dense(), H.to_dense())

We can also get the Group of Translations or other symmetries of this lattice, if you're into group theory and simmetries...  but we won't be talking about that

In [ ]:
lattice.translation_group()

## 2. Exact Diagonalization

Now that we have defined the Hamiltonian, we can already start playing with it. For example, since the number of spins is large but still manageable for exact diagonalization, we can give it a try. 

In NetKet this is easily done converting our Hamiltonian operator into a sparse matrix of size $ 2^N \times 2^ N $. 

In [ ]:
sp_h=H.to_sparse()
sp_h.shape

Since this is just a regular scipy sparse matrix, we can just use any sparse diagonalization routine in there to find the eigenstates. For example, this will find the two lowest eigenstates  

In [ ]:
from scipy.sparse.linalg import eigsh

eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")

print("eigenvalues with scipy sparse:", eig_vals)

E_gs = eig_vals[0]

## 3. Variational States: A RBM-NQS

In NetKet one has to define a variational function approximating the **logarithm** of the wave-function amplitudes (or density-matrix values).
We call this variational function _the Model_ (yes, caps on the M).

$$ \langle \sigma^{z}_1,\dots \sigma^{z}_N| \Psi_{\mathrm{mf}} \rangle = \exp\left[\mathrm{Model}(\sigma^{z}_1,\dots \sigma^{z}_N ; \theta ) \right], $$

where $\theta$ is a set of parameters. 

The Model can be defined using one of the several *functional* jax frameworks such as Jax/Stax, Flax or Haiku. 
NetKet includes several pre-built models and layers built with [Flax](https://github.com/google/flax), so we will be using it for the rest of the notebook. 

In [ ]:
# This only creates the 'definition' of the model, not the parameters themselves.
model = nk.models.RBM(alpha=1, param_dtype=complex)

The model itself is only a set of instructions on how to initialise the parameters and how to compute the result. 

In [ ]:
# To initialize the parameters, you provide some 'mock' inputs:
pars = model.init(jax.random.PRNGKey(0), hi.numbers_to_states(np.arange(3)))

# and then we can evaluate it:
input = hi.random_state(jax.random.PRNGKey(3), 10)
logpsi_val = model.apply(pars, input)
print(f"An input of shape {input.shape=} gives output {logpsi_val.shape=}")

### 3a. Constructing the Variational State

To actually create a variational state with its parameters, the easiest way is to construct a Monte-Carlo-sampled Variational State. 
To do this, we first need to define a sampler.

In `netket.sampler` several samplers are defined, each with its own peculiarities. 
In the following example, we will be using a simple sampler that flips the spins in the configurations one by one.

You can read more about how the sampler works by checking the documentation with `?nk.sampler.MetropolisLocal`

In [ ]:
# Create the local sampler on the hilbert space
sampler = nk.sampler.MetropolisLocal(hi, n_chains=8, n_sweeps=1)

# Construct the variational state using the model and the sampler above.
# n_samples specifies how many samples should be used to compute expectation
# values.
vstate = nk.vqs.MCState(sampler, model, n_samples=20*4000)

You can play around with the variational state: for example, you can compute expectation values yourself or inspect it's parameters.
The parameters are stored as a set of nested dictionaries. In this case, the single parameter $\lambda$ is stored inside a (frozen) dictionary.
(The reason why the dictionary is frozen is a detail of Flax).

In [ ]:
print(vstate)
print(vstate.parameters)

### 3b. Computing expectation values

The **AllMighty** Giuseppe Carleo has shown you that, given a variational state

$$
\vert\psi_\theta\rangle = \sum_\sigma \psi_\theta(\sigma)\vert\sigma\rangle
$$

We can estimate expectation values in Polynomial time by writing them as:

\begin{align}
\langle \psi_\theta\vert \hat{H}\vert\psi_\theta \rangle &= \sum_\sigma \langle \psi_\theta\vert\sigma\rangle\langle\sigma\vert \hat{H}\vert\psi_\theta \rangle \\
&=  \sum_\sigma \langle \psi_\theta\vert\sigma\rangle\langle\sigma\vert\psi_\theta\rangle\frac{\langle\sigma\vert \hat{H}\vert\psi_\theta \rangle}{\langle\sigma\vert\psi_\theta\rangle} \\
&=  \sum_\sigma |\psi_\theta(\sigma)|^2H^{\text{Loc}}(\sigma) \\
&= \mathbb{E}_{\sigma\sim|\psi_\theta(\sigma)|^2}\left[H^{\text{Loc}}(\sigma)\right]
\end{align}

In [ ]:
# let's generate the samples
σ = vstate.sample()
print(f"My samples have {σ.shape = } which comes from {vstate.sampler.n_chains = } ",
      "chains of 10'000 samples each, and an hilbert space of {hi.size=} spins.")

# we can also inspect the acceptance rate:
print("The acceptance rate right now is stored in the `sampler_state`: ", vstate.sampler_state)

In [ ]:
# Let's generate the local energies
H_loc = vstate.local_estimators(H)
print(f"My local estimators have a shape of {H_loc.shape = }. The samples are taken from `vstate.samples`, which are the last sampled samples.")

In [ ]:
# Let's compute the expectation value:
print("<H> =", jnp.mean(H_loc), "Var[H] = ", jnp.var(H_loc), "\n")

# But a much better way to compute it would be: 
nk.stats.statistics(H_loc)

With a variational state, you can compute expectation values directly of operators using `vstate.expect`. For our `MCState`, this internally performs Monte Carlo sampling over `n_samples` spin configurations.

In [ ]:
vstate.n_samples = 4000
E = vstate.expect(H)
print(type(E))
print(E)

The return value of `.expect` is an object of type `Stats` which, beside the energy expectation value contains some statistics of the ensemble of local energies over which the energy is estimated.

The `Stats` object also has a dictionary representation:

In [ ]:
print(E.to_dict())

The `Stats` object contains the following diagnostics:

* The `Mean` over the local energy samples, which is an estimate of the quantum expectation value $\langle \hat H \rangle.$

* The `Variance` over the local energy samples, which is an estimate of the quantum variance $\langle(\delta\hat H)^2\rangle = \langle (\hat H - \langle \hat H \rangle)^2 \rangle.$

* The Monte Carlo standard error (MCSE) of the mean (as `Sigma`).

* An estimate `TauCorr` of the autocorrelation time over the Markov chains.

* `R_hat`, which is the so-called Gelman-Rubin split-R_hat diagnostic, which indicates whether the MCMC chain is converged (see below).

If you are close to an eigenstate of the operators, the variance should be close to 0. (In an exact eigenstate, the local energy is constant and therefore the variance is exactly zero.)

The Gelman-Rubin diagnostic will be $\hat{R}\approx 1$ if the Markov chains are converged, while it will be larger than $1$ if your sampling has not converged.
As a rule of thumb, look out for $|\hat{R}| > 1.1$, and if that occurs consistently, check if you need more samples or if you MCMC scheme is even consistent with your system. (This is a somewhat weak criterion. Modern MCMC literature tends to recommend $|\hat{R}| > 1.01$ to discard a sample. NQS optimization can still work with the less stringent criterion, but if you see stability problems, keep this in mind.)

You can also investigate the correlation time of your estimator, $\tau$. If $\tau\gg1$ then your samples are very correlated and you most likely have some issues with your sampling scheme.

You can also access the fields individually:
Note that if you run your calculation using MPI on different processes/machines, those estimators will return the mean, error and estimators of all the samples across all the processes.

### 3c. Computing gradients of Expectation values

Above we have computed the expectation value as:

\begin{align}
\langle \psi_\theta\vert \hat{H}\vert\psi_\theta \rangle &=   \sum_\sigma |\psi_\theta(\sigma)|^2H^{\text{Loc}}(\sigma) \\
&= \mathbb{E}_{\sigma\sim|\psi_\theta(\sigma)|^2}\left[H^{\text{Loc}}(\sigma)\right]
\end{align}

But... in order to use an optimisation algorithm such as gradient descent, we will need to compute the gradient of this quantity as well. 
For an expectation value, it is easy to derive the formula:

$$
    \nabla_\theta \mathbb{E}_{X\sim p_\theta(X)}\left[ f_\theta(X) \right] = \mathbb{E}_{X\sim p_\theta(X)}\left[\nabla_\theta f_\theta(X) \right]- \mathbb{E}_{X\sim p_\theta(X)}\left[(\nabla_\theta\log\psi_\theta(X))f_\theta(X) \right]
$$

Which, translated to our case, gives:

\begin{align}
\nabla_\theta \langle \psi_\theta\vert \hat{H}\vert\psi_\theta \rangle = \\
&= 2\Re\left\{\mathbb{E}_{\sigma\sim|\psi_\theta(\sigma)|^2}\left[(\nabla_\theta\log\psi(\sigma)) H^{\text{Loc}}(\sigma)\right] - \mathbb{E}_{\sigma\sim|\psi_\theta(\sigma)|^2}\left[\nabla_\theta\log\psi(\sigma)\right] \mathbb{E}_{\sigma\sim|\psi_\theta(\sigma)|^2}\left[H^{\text{Loc}}(\sigma)\right] \right\} \,\,\,\, &&\text{If parameters are real} \\
&= \mathbb{E}_{\sigma\sim|\psi_\theta(\sigma)|^2}\left[(\nabla_\theta\log\psi(\sigma)) H^{\text{Loc}}(\sigma)\right] - \mathbb{E}_{\sigma\sim|\psi_\theta(\sigma)|^2}\left[\nabla_\theta\log\psi(\sigma)\right] \mathbb{E}_{\sigma\sim|\psi_\theta(\sigma)|^2}\left[H^{\text{Loc}}(\sigma)\right] \,\,\,\, &&\text{If parameters are complex}
\end{align}

Now, as we showed before, the parameters in NetKet are not stored as a single array/vector $\vec{\theta} = \{\theta_1, \theta_2, \dots\theta_N\}$, but rather as a PyTree, which is a set of nested dictionaries containing arbirarily shaped matrices, tensors and scalars which, overall, behave as arrays:

In [ ]:
jax.tree_map(lambda x: (x.shape, x.dtype), vstate.parameters)

So we will want the gradient to have the same shape...

I will not show you how to do it by hand (because it's a bit involved, but you can have a look [here](https://github.com/netket/netket/blob/77a23dcdc42b63f0116510e1064e82351f830fb0/netket/vqs/mc/mc_state/expect_forces.py#L67) if you really are curious.
Instead, NetKet allows you to compute the gradient directly and efficiently by calling the function 

```
vstate.expect_and_grad(operator)
```

In [ ]:
E, grad_E = vstate.expect_and_grad(H)

jax.tree_map(lambda x: (x.shape, x.dtype), grad_E)

Lets look at how the statistics work a bit more closely. We'll be using some artifically generated random data:

Getting back to our VMC example, let's try out a couple of `n_samples` values and see what happens to the energy statistics:

## 4. Variational Monte Carlo

We will now try to optimise $ \lambda $ in order to best approximate the ground state of the hamiltonian.

At first, we'll try to do this by ourself by writing the training loop, but then we'll switch to using a pre-made
solution provided by netket for simplicity.

### 4a. DIY Optimisation loop

The optimisation (or training) loop must do a very simple thing: at every iteration it must compute the energy and it's gradient, then multiply the gradient by a certain learning rate $\lambda = 0.05$ and lastly it must update the parameters with this rescaled gradient.
You can do so as follows:

In [ ]:
from tqdm import tqdm

energy_history = []
n_steps = 100

# For every iteration (tqdm is just a progress bar)
for i in tqdm(range(n_steps)):
    # compute energy and gradient of the energy
    E, E_grad = vstate.expect_and_grad(H)
    # log the energy to a list
    energy_history.append(E.mean.real)
    # equivalent to vstate.parameters - 0.05*E_grad , but it performs this
    # function on every leaf of the dictionaries containing the set of parameters
    new_pars = jax.tree_map(lambda x,y: x-0.05*y, vstate.parameters, E_grad)
    # actually update the paramters
    vstate.parameters = new_pars

We now can plot the energy during those optimisation steps:


In [ ]:
plt.plot(energy_history)

### 4b. Use NetKet's optimisation driver 

As writing the whole optimisation loop by yourself every time is.. boring, we can make use of a coupled of NetKet's built-in utilities.

In [ ]:
# First we reset the parameters to run the optimisation again
vstate.init_parameters()

# Then we create an optimiser from the standard library.
# You can also use optax.
optimizer = nk.optimizer.Sgd(learning_rate=0.1)

# build the optimisation driver
gs = nk.driver.VMC(H, optimizer, variational_state=vstate)

# Use one of the NetKet's internal loggers
log = nk.logging.RuntimeLog()

# run the driver for 300 iterations. This will display a progress bar
# by default.
gs.run(n_iter=300, out=log)

mf_energy=vstate.expect(H)
error=abs((mf_energy.mean-eig_vals[0])/eig_vals[0])
print("Optimized energy and relative error: ",mf_energy,error)

In [ ]:
eig_vals[0]

In [ ]:
log.data['Energy']

In [ ]:
plt.errorbar(log.data['Energy'].iters, log.data['Energy'].Mean.real, yerr=log.data['Energy'].Sigma);
plt.xlabel("Iteration")
plt.ylabel("Energy")
#plt.yscale('log')

## 5. Dynamics! 

Let's try to do dynamics. Starting from the ground state at $h=1$ . 

To be safe, let's store our starting parameters (so that we can always reset them).
In Jax/Flax/NetKet arrays and parameters are immutable, meaning that you can **never** modify them in-place. So storing them here is safe.

In [ ]:
θ_0 = vstate.parameters

Now let's define the parameters of the simulation we want to run, so that we can generate some exact data to benchmark against

In [ ]:
H_quench = nk.operator.Ising(hi, lattice, h=0.5, J=1.0)

t0 = 0.0
t_end = 2.0

# observables
Sx = sum([nk.operator.spin.sigmax(hi, i) for i in range(hi.size)])/hi.size

### 5a. Exact solution

Let's now use [QuTiP](https://qutip.org/) to solve the equation exactly.
To convert NetKet's objects to QuTiP you can just call `.to_qobj()` on them. Remember that QuTiP scales exponentially, as it does calculations in the full Hilbert space.

In [ ]:
import qutip as qt

#Schroedinger-Equation-SOLVE
sol = qt.sesolve(H_quench.to_qobj(), vstate.to_qobj(), np.arange(t0, t_end, 0.01), e_ops=[Sx.to_qobj()])

In [ ]:
plt.plot(sol.times, sol.expect[0])
plt.xlabel("Time t")
plt.ylabel("Sigma X")

### 5b. TDVP

Let's now use the TDVP.

You have heard in the lecture by the AllMighty how to do time evolution on a variational ansatz.
Otherwise, helpful references for the derivation of the TDVP equations of motion are, e.g, [Yuan et al. "Theory of Variational Simulation" (Quantum 3, 191, 2019)](https://quantum-journal.org/papers/q-2019-10-07-191/), and [Stokes et al. (arXiv:2203.14824)](https://arxiv.org/abs/2203.14824).

Assuming we have the parametrized variational ansatz $\vert\psi_\theta\rangle $, where to every set of parameters $\theta$ we associate the state $\theta \rightarrow \vert\psi_\theta\rangle$, determining the dyanmics will mean finding the parameters at every time $\theta(t)$ so that
$$
    \theta(t) \rightarrow \vert\psi_{\theta(t)}\rangle
$$

Looking at a single time-step, assuming we know the parameters $\theta(t)$, we want to determine the parameters at time $\theta(t+dt)$. 
Those must maximise the overlap between the evolved-state $\vert \psi_{\theta(t+dt)}\rangle$ and the _exactly evolved_ $ e^{-i\hat{H}dt}\vert\psi_{\theta(t)}\rangle$

$$
    \max_{\delta\theta} |\langle \mathrm{e}^{-\gamma \hat H \delta t} \psi_{\theta} | \psi_{\theta + \delta\theta} \rangle|^2
$$


Taylor expanding this condition to second order in $\delta\theta$ and $\delta t$ yields after some steps the equation of motion

$$
    S(\theta) \, \dot\theta = -\gamma F(\theta, t)
$$

with the quantum geometric tensor

$$
    S_{ij}(\theta) = \frac{
       \langle\partial_i\psi_\theta | \partial_j\psi_\theta\rangle
    }{
       \langle \psi_\theta | \psi_\theta \rangle
    } - \frac{
       \langle\partial_i\psi_\theta | \psi_\theta \rangle\langle \psi_\theta | \partial_j\psi_\theta\rangle
    }{
       \langle \psi_\theta | \psi_\theta \rangle^2
    }
$$

and forces vector (Note: this is the same as the gradient of the expectation value if $\theta_i\in\mathbb{C}$, but it **is not** the same if the parameters are Real. In particular, if parameters are real, the gradient of the expectation value is the real part of the force vector shown below)

$$
    F_i(\theta, t) = \frac{\partial\langle \hat H \rangle}{\partial\theta_i^*}
$$


$\gamma = 1$ results in imaginary time evolution, $\gamma = \mathrm i$ gives real time evolution instead.

$S$ and $F$ can be estimated using Monte Carlo sampling: Given $\sigma \sim |\psi_\theta(\sigma)|^2$, we can estimate those quantities by
$$
    S_{ij} = \mathbb{C}\text{ov}_{\sigma\sim|\psi_\theta(\sigma)|^2}(O_i(\sigma), O_j(\sigma))
    \qquad
    F_i = \mathbb{C}\text{ov}_{\sigma\sim|\psi_\theta(\sigma)|^2}(O_i(\sigma), H^{\text{Loc}}(\sigma))
$$

where $O_j$ are the log-derivatives

$$
    O_j(\sigma) = \frac{\partial\ln\psi_\theta(\sigma)}{\partial\theta_j}.
$$

In NetKet, the quantum geometric tensor $S_\theta$ is available from the variational state class:

In [ ]:
vstate.quantum_geometric_tensor()

This object behaves like a matrix that can be multiplied by the vector of parameters:

In [ ]:
#vstate.quantum_geometric_tensor() @ vstate.parameters

So the only thing we need is now to implement the equations above:

In [ ]:
# The geometric tensor
S = vstate.quantum_geometric_tensor()

# The forces vector
E, F = vstate.expect_and_forces(H_quench)

# multiply by -1j
gamma_F = jax.tree_map(lambda x: -1j*x, F)

# solve the linear system dθ = S^{-1}F
dθ, _ = S.solve(nk.optimizer.solver.svd, gamma_F)

dt = 0.01
θ_tpdt = jax.tree_map(lambda x, y: x + dt * y, vstate.parameters, dθ)

Let's put this in a function that takes the current state and returns also some observable:

In [ ]:
def timestep(vstate, dt):
    # The geometric tensor
    S = vstate.quantum_geometric_tensor()

    # The forces vector
    E, F = vstate.expect_and_forces(H_quench)

    # multiply by -1j
    gamma_F = jax.tree_map(lambda x: -1j*x, F)

    # solve the linear system dθ = S^{-1}F
    dθ, _ = S.solve(nk.optimizer.solver.svd, gamma_F)

    θ_tpdt = jax.tree_map(lambda x, y: x + dt * y, vstate.parameters, dθ)
    return E, θ_tpdt

And we can use this function in a loop to solve the full dynamics, storing the Energy and other quantities at every timestep.

In [ ]:
t = t0
vstate.parameters = θ_0
dt = 0.005

log_dynamics = nk.logging.RuntimeLog()

for t in tqdm(np.arange(t0, t_end, dt)):
    # Take one time-step
    E_t, θ_tpdt = timestep(vstate, dt)
    
    # Put together a dictionary of quantities to 'log' at this timestep
    log_data_t = {'Energy': E_t, 'Sx': vstate.expect(Sx)}
    # Log the data
    log_dynamics(t, log_data_t, vstate)
    
    # update the parameters
    vstate.parameters = θ_tpdt
    #t = t + dt

In [ ]:
plt.plot(log_dynamics.data['Energy'].iters, log_dynamics.data['Energy'].Mean.real)

In [ ]:
plt.plot(sol.times, sol.expect[0], label = "Exact")
plt.plot(log_dynamics.data['Energy'].iters, log_dynamics.data['Sx'].Mean.real, label="RBM(alpha=1)+dt=0.005")
plt.ylabel("Sigma X")
plt.xlabel("Time t")
plt.legend();


### 5c. NetKet's simpler interface

Instead of writing your time-integration loop, or if you want to use higher-order ODE integrators (beyond Euler) to solve the differential equation, you can use our own TDVP interface:

In [ ]:
# It's in netket.experimental, because we are still not 100% sure we got the API right, but it has been stable for the last year or so
import netket.experimental as nkx

In [ ]:
# Pick an integrator (this has order 2)
integrator = nkx.dynamics.Heun(dt=0.001)

In [ ]:
# We reset the parameters again
vstate.parameters = θ_0

driver = nkx.TDVP(
    H_quench,
    vstate,
    integrator,
    linear_solver=nk.optimizer.solver.svd,
    qgt=nk.optimizer.qgt.QGTJacobianDense(holomorphic=True),
)

In [ ]:
log = nk.logging.RuntimeLog()
driver.run(T=1.0, obs={"Sx": Sx}, out=log)